# testS
Here I put some test that was sussexfull with all my reasoning


(1) here we can select the points that are the vertices of the enviroment to calibrate the camera and then work correctly!


(2) here I'm trying to use the webcam to do detection of the symbol on the thymio and the obstacle in the enviroment
--> I notice that the thymio identification works quite well but for the obstacle no. Maybe we should reason better at the design choices and at the filtering we can do

In [10]:
# (1)
import cv2
import numpy as np

# --- 1. CONFIGURATION ---

# Set to True to use the webcam, False to use a static image
USE_WEBCAM = True

# Your Aukey webcam is likely index 0, but it could be 1, 2, etc.
# Try changing this if 0 doesn't work.
CAMERA_INDEX = 0 

# Path to your static image (if USE_WEBCAM is False)
IMAGE_PATH = "prova.jpg" 

# Desired output resolution for your top-down map
# A 4:3 ratio is common, but you can change this
MAP_WIDTH = 800
MAP_HEIGHT = 600

# --- 2. GLOBAL VARIABLES ---

# List to store the 4 clicked points (source points)
src_points = []
# The image we will use for picking points
calibration_frame = None

# --- 3. MOUSE CALLBACK FUNCTION ---

def click_event(event, x, y, flags, params):
    """
    Handles mouse clicks. 
    Saves the (x, y) coordinates of 4 clicks.
    """
    global src_points, calibration_frame
    
    # Check if the left mouse button was clicked
    if event == cv2.EVENT_LBUTTONDOWN:
        
        if len(src_points) < 4:
            # Add the clicked point to the list
            src_points.append((x, y))
            
            # Draw a circle on the image to show feedback
            cv2.circle(calibration_frame, (x, y), 5, (0, 255, 0), -1)
            cv2.imshow("Source Image - Click 4 Corners", calibration_frame)
            print(f"Point {len(src_points)} added: ({x}, {y})")
        
        if len(src_points) == 4:
            print("All 4 points selected. Press 'c' to calculate the transform.")

# --- 4. MAIN SCRIPT ---

# --- Step 4a: Get the image for calibration ---

if USE_WEBCAM:
    print(f"Opening webcam index {CAMERA_INDEX}...")
    cap = cv2.VideoCapture(CAMERA_INDEX)
    
    # Optional: Set high resolution for your 1080p camera
    # This might slow down processing, but gives better calibration
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
    
    if not cap.isOpened():
        print(f"Error: Could not open webcam index {CAMERA_INDEX}.")
        print("Try changing CAMERA_INDEX to 1 or 2.")
        exit()

    print("\nWebcam open.")
    print("Press 's' to snapshot the current frame for calibration.")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Can't receive frame. Exiting...")
            exit()
        
        # Show a small preview
        preview = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        cv2.imshow("Webcam Preview - Press 's' to snapshot", preview)
        
        if cv2.waitKey(1) & 0xFF == ord('s'):
            calibration_frame = frame.copy()
            print("Snapshot taken! Please close the preview window.")
            cv2.destroyWindow("Webcam Preview - Press 's' to snapshot")
            break
else:
    print(f"Loading image from {IMAGE_PATH}...")
    calibration_frame = cv2.imread(IMAGE_PATH)
    if calibration_frame is None:
        print(f"Error: Could not load image from {IMAGE_PATH}.")
        exit()
    print("Image loaded.")

# --- Step 4b: Select 4 corners ---

# Create a window and set the mouse callback
cv2.namedWindow("Source Image - Click 4 Corners")
cv2.setMouseCallback("Source Image - Click 4 Corners", click_event)

print("\n--- INSTRUCTIONS ---")
print("Click on the 4 corners of your robot arena in this order:")
print("  1. Top-Left")
print("  2. Top-Right")
print("  3. Bottom-Right")
print("  4. Bottom-Left")
print("After 4 clicks, press 'c' to continue.")

cv2.imshow("Source Image - Click 4 Corners", calibration_frame)

# Wait until 'c' is pressed
while True:
    if cv2.waitKey(1) & 0xFF == ord('c'):
        if len(src_points) == 4:
            break
        else:
            print("Please click exactly 4 points before pressing 'c'.")

cv2.destroyWindow("Source Image - Click 4 Corners")

# --- Step 4c: Calculate Transform Matrix ---

print("Calculating perspective transform matrix...")

# Convert points to NumPy array in float32 format
src_points_np = np.float32(src_points)

# Define the 4 destination points (the corners of our output map)
dst_points_np = np.float32([
    [0, 0],                  # Top-Left
    [MAP_WIDTH, 0],          # Top-Right
    [MAP_WIDTH, MAP_HEIGHT], # Bottom-Right
    [0, MAP_HEIGHT]          # Bottom-Left
])

# Calculate the perspective transform matrix
matrix = cv2.getPerspectiveTransform(src_points_np, dst_points_np)
print("Matrix calculated successfully!")
np.save("my_matrix.npy", matrix)  #MAYBE NOT HERE


# --- Step 4d: Apply Transform in a Loop ---

print("\nStarting the real-time top-down feed.")
print("Press 'q' in the 'Top-Down Map' window to quit.")

try:
    while True:
        if USE_WEBCAM:
            # Read a new frame from the webcam
            ret, frame = cap.read()
            if not ret:
                print("Webcam disconnected.")
                break
            
            # Apply the perspective transform
            top_down_map = cv2.warpPerspective(frame, matrix, (MAP_WIDTH, MAP_HEIGHT))
            
        else:
            # For a static image, just apply it once
            top_down_map = cv2.warpPerspective(calibration_frame, matrix, (MAP_WIDTH, MAP_HEIGHT))
        
        # Display the result
        cv2.imshow("Top-Down Map", top_down_map)
        
        # Break loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        # If using a static image, we don't need to loop.
        if not USE_WEBCAM:
            print("Static image transform complete. Press 'q' to quit.")
            cv2.waitKey(0) # Wait forever until a key is pressed
            break

finally:
    # --- 5. CLEANUP ---
    print("Cleaning up and closing windows.")
    if USE_WEBCAM:
        cap.release()
    cv2.destroyAllWindows()

Opening webcam index 0...

Webcam open.
Press 's' to snapshot the current frame for calibration.
Snapshot taken! Please close the preview window.

--- INSTRUCTIONS ---
Click on the 4 corners of your robot arena in this order:
  1. Top-Left
  2. Top-Right
  3. Bottom-Right
  4. Bottom-Left
After 4 clicks, press 'c' to continue.
Point 1 added: (381, 236)
Point 2 added: (797, 210)
Point 3 added: (768, 576)
Point 4 added: (384, 568)
All 4 points selected. Press 'c' to calculate the transform.
Calculating perspective transform matrix...
Matrix calculated successfully!

Starting the real-time top-down feed.
Press 'q' in the 'Top-Down Map' window to quit.
Cleaning up and closing windows.


In [11]:
import cv2
import numpy as np
import math

# --- 1. CONFIGURATION ---

# Set to True to use the webcam, False to use a static image
USE_WEBCAM = True  # <-- CHANGE THIS TO SWITCH

# Your Aukey webcam is likely index 0
CAMERA_INDEX = 0

# Path to your static image (if USE_WEBCAM is False)
IMAGE_PATH = "my_arena_image.jpg"  # <-- SET YOUR TEST IMAGE PATH

# --- CONFIGURATION (from Step 1) ---
MAP_WIDTH = 800
MAP_HEIGHT = 600

# --- ARUCO CONFIGURATION ---
ARUCO_DICT = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
ARUCO_PARAMETERS = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(ARUCO_DICT, ARUCO_PARAMETERS)

THYMIO_MARKER_ID = 0  # ID of the ArUco marker on your Thymio
GOAL_MARKER_ID = 1    # ID of the ArUco marker for the goal

# --- OBSTACLE CONFIGURATION (HSV for WHITE) ---
# White in HSV has:
# - Hue: 0-180 (irrelevant)
# - Saturation: 0-25 (very low)
# - Value: 200-255 (very high)
# These are the ranges we'll use to create a mask.
LOWER_WHITE_HSV = np.array([0, 0, 200])
UPPER_WHITE_HSV = np.array([180, 25, 255])
MIN_OBSTACLE_AREA = 100  # Minimum contour area to be considered an obstacle

# --- GLOBAL VARIABLES (for visualization/debugging) ---
thymio_pose = None
goal_position = None
obstacle_contours = []

# --- FUNCTIONS ---

def detect_aruco_markers(frame):
    """
    Detects ALL ArUco markers and returns their poses in a dictionary
    keyed by marker ID.
    Returns: {id: ((x, y), angle_deg), ...}
    """
    corners, ids, rejected = detector.detectMarkers(frame)
    
    poses = {}

    if ids is not None:
        for i, marker_id in enumerate(ids.flatten()):
            marker_corners = corners[i][0]
            
            # Calculate center
            center_x = int(np.mean(marker_corners[:, 0]))
            center_y = int(np.mean(marker_corners[:, 1]))
            
            # Calculate orientation
            # ArUco corners order: Top-left, Top-right, Bottom-right, Bottom-left
            bottom_mid_x = int((marker_corners[2][0] + marker_corners[3][0]) / 2)
            bottom_mid_y = int((marker_corners[2][1] + marker_corners[3][1]) / 2)
            
            top_mid_x = int((marker_corners[0][0] + marker_corners[1][0]) / 2)
            top_mid_y = int((marker_corners[0][1] + marker_corners[1][1]) / 2)
            
            dx = top_mid_x - bottom_mid_x
            dy = top_mid_y - bottom_mid_y
            
            angle_rad = math.atan2(dx, -dy) 
            angle_deg = math.degrees(angle_rad)
            if angle_deg < 0:
                angle_deg += 360
                
            poses[marker_id] = ((center_x, center_y), angle_deg)
    
    return poses

def detect_obstacles_hsv(frame):
    """
    Detects white obstacles on a colored background using HSV color space.
    Returns the raw contours.
    """
    # 1. Convert frame to HSV
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # 2. Create a binary mask for "white"
    mask = cv2.inRange(hsv_frame, LOWER_WHITE_HSV, UPPER_WHITE_HSV)
    
    # 3. Clean up the mask (optional but recommended)
    #    'MORPH_OPEN' removes small white noise dots (erosion then dilation)
    kernel = np.ones((5, 5), np.uint8)
    mask_cleaned = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    # 4. Find contours on the cleaned mask
    contours, _ = cv2.findContours(mask_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 5. Filter contours by area
    valid_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > MIN_OBSTACLE_AREA:
            valid_contours.append(contour)
    
    return valid_contours, mask_cleaned # Return mask for debugging

# --- MAIN SCRIPT ---

# --- Load Perspective Transform Matrix ---
# You MUST replace this with your actual matrix from Step 1
# e.g., matrix = np.load("my_matrix.npy")
matrix = np.load("my_matrix.npy")
#matrix = np.array([
    #[1.0, 0.0, 0.0],
    #[0.0, 1.0, 0.0],
    #[0.0, 0.0, 1.0]
#], dtype=np.float32)
#print("NOTE: Using a DUMMY IDENTITY MATRIX. Replace with your actual perspective transform matrix!")
# --- END DUMMY MATRIX ---


# --- Initialize Video Capture or Load Image ---
cap = None
if USE_WEBCAM:
    cap = cv2.VideoCapture(CAMERA_INDEX)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
    if not cap.isOpened():
        print(f"Error: Could not open webcam index {CAMERA_INDEX}.")
        exit()
    print("\nStarting localization from webcam. Press 'q' to quit.")
else:
    print(f"\nStarting localization from image: {IMAGE_PATH}")

try:
    while True:
        frame = None
        if USE_WEBCAM:
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame from webcam. Exiting...")
                break
        else:
            # Load the static image
            frame = cv2.imread(IMAGE_PATH)
            if frame is None:
                print(f"Error: Could not load image from {IMAGE_PATH}.")
                break
        
        # 1. Apply Perspective Transform (from Step 1)
        top_down_map = cv2.warpPerspective(frame, matrix, (MAP_WIDTH, MAP_HEIGHT))

        # 2. Detect Thymio and Goal
        all_poses = detect_aruco_markers(top_down_map)
        
        # Reset values
        thymio_pose = None
        goal_position = None
        
        if THYMIO_MARKER_ID in all_poses:
            thymio_pose = all_poses[THYMIO_MARKER_ID]
        
        if GOAL_MARKER_ID in all_poses:
            goal_position = all_poses[GOAL_MARKER_ID][0] # We only need the (x,y) position

        # 3. Detect Obstacles
        obstacle_contours, obstacle_mask = detect_obstacles_hsv(top_down_map)

        # --- VISUALIZATION (for debugging) ---
        display_frame = top_down_map.copy()

        # Draw Thymio
        if thymio_pose is not None:
            pos, angle = thymio_pose
            cv2.circle(display_frame, pos, 10, (0, 255, 255), -1) # Yellow
            angle_rad = math.radians(angle)
            end_x = int(pos[0] + 30 * math.sin(angle_rad))
            end_y = int(pos[1] - 30 * math.cos(angle_rad))
            cv2.arrowedLine(display_frame, pos, (end_x, end_y), (0, 255, 255), 2)
            cv2.putText(display_frame, f"Thymio (ID {THYMIO_MARKER_ID})", 
                        (pos[0] + 15, pos[1]), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

        # Draw Goal
        if goal_position is not None:
            cv2.circle(display_frame, goal_position, 15, (255, 0, 0), -1) # Blue
            cv2.putText(display_frame, f"Goal (ID {GOAL_MARKER_ID})", (goal_position[0] + 15, goal_position[1]), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        
        # Draw Obstacles
        # This draws the PRECISE outlines you asked for
        cv2.drawContours(display_frame, obstacle_contours, -1, (0, 0, 255), 2) # Red outlines
        
        # Display the main frame
        cv2.imshow("Top-Down Map with Detections", display_frame)
        
        # Display the binary mask for tuning/debugging
        cv2.imshow("Obstacle Mask (Debug)", obstacle_mask)

        # --- Loop Control ---
        if USE_WEBCAM:
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            print("Detection complete. Press any key to quit.")
            cv2.waitKey(0)
            break

finally:
    # --- CLEANUP ---
    print("Cleaning up and closing windows.")
    if USE_WEBCAM and cap is not None and cap.isOpened():
        cap.release()
    cv2.destroyAllWindows()


Starting localization from webcam. Press 'q' to quit.
Cleaning up and closing windows.


In [5]:
import cv2
import numpy as np

# --- Configuration ---

# Define the dictionary. We'll use the same one from the detection code.
# DICT_4X4_50 is a good choice: 4x4 squares, 50 unique IDs.
ARUCO_DICT_NAME = cv2.aruco.DICT_4X4_50

# The ID you want to generate.
MARKER_ID = 1

# The size of the output image in pixels (e.g., 300x300).
# Make this large enough for a high-quality print.
IMAGE_SIZE = 300

# The name of the output file.
OUTPUT_FILE = f"aruco_marker_id_{MARKER_ID}.png"

# --- Generation ---

# 1. Get the ArUco dictionary
try:
    aruco_dict = cv2.aruco.getPredefinedDictionary(ARUCO_DICT_NAME)
except AttributeError:
    print(f"Error: Unable to find ArUco dictionary. Make sure you have 'opencv-python-contrib' installed.")
    exit()

# 2. Create an empty image (numpy array)
# Note: ArUco functions in older OpenCV versions might draw directly.
# The modern way is to use `generateImageMarker`.
print(f"Generating marker with ID={MARKER_ID} from dictionary {ARUCO_DICT_NAME}...")

# 3. Generate the marker
# This function creates the marker image directly.
# The '1' argument is the border size (in bits/modules). 1 is standard.
marker_image = cv2.aruco.generateImageMarker(aruco_dict, MARKER_ID, IMAGE_SIZE, borderBits=1)

if marker_image is not None:
    # 4. Save the image to a file
    cv2.imwrite(OUTPUT_FILE, marker_image)
    print(f"Successfully saved marker to '{OUTPUT_FILE}'")
    
    # Optional: Display the marker
    cv2.imshow("ArUco Marker", marker_image)
    print("Press any key to close the preview.")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Error: Could not generate ArUco marker.")

Generating marker with ID=1 from dictionary 0...
Successfully saved marker to 'aruco_marker_id_1.png'
Press any key to close the preview.
